# Week 3 — Limits & Colimits (Micro‑Demos)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sridharmahadevan/Category-Theory-for-AGI-UMass-CMPSCI-692CT/blob/main/notebooks/week03_limits_colimits_microdemo.ipynb)
<br/>
_Replace `sridharmahadevan/Category-Theory-for-AGI-UMass-CMPSCI-692CT` above once you push this repo to GitHub._

### Environment (run first)
This pins a minimal, stable stack. GPU is **optional**; notebooks run on CPU.

In [ ]:
%%capture
# Core scientific stack + causal / graph tooling
%pip install -q numpy==1.* pandas==2.* matplotlib==3.* networkx==3.* pgmpy==0.1.* graphviz==0.20.*
# Torch CPU by default (Colab often preinstalls a GPU build; this is a safe fallback)
%pip install -q torch --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
# Install system graphviz only if available (Colab & many Linux envs). Safe to skip elsewhere.
!command -v apt-get >/dev/null && apt-get -y -qq install graphviz || echo "apt-get not available; skipping system graphviz"

In [ ]:
import platform, sys
print("Python:", platform.python_version())
try:
    import torch
    print("Torch:", torch.__version__, "| CUDA available?", torch.cuda.is_available())
    device = "cuda" if torch.cuda.is_available() else "cpu"
except Exception as e:
    print("Torch not installed, proceeding CPU-only.")
    device = "cpu"
device


## Learning goals
- Compute a **pullback** (limit) in **Set** for two functions \(X\xrightarrow{f} Z\xleftarrow{g} Y\).
- Compute a **pushout** (colimit) in **Set** for \(X\xleftarrow{u} W\xrightarrow{v} Y\) as quotient (gluing).


## Pullback demo

In [ ]:

X = {"x0","x1","x2"}
Y = {"y0","y1"}
Z = {"z0","z1"}

f = {"x0":"z0", "x1":"z1", "x2":"z1"}  # X→Z
g = {"y0":"z1", "y1":"z0"}             # Y→Z

pullback = {(x,y) for x in X for y in Y if f[x] == g[y]}
print("Pullback X×_Z Y =", pullback)  # pairs with matching images


## Pushout demo

In [ ]:

# W injects into X and Y via u and v. Identify points with same preimage.
W = {"w0","w1"}
X = {"x0","x1","x2"}
Y = {"y0","y1"}
u = {"w0":"x1", "w1":"x2"}   # W→X
v = {"w0":"y0", "w1":"y1"}   # W→Y

# Disjoint union X ⊔ Y represented as tagged elements
disjoint = {("X",x) for x in X} | {("Y",y) for y in Y}

# Equivalence relation generated by u(w) ~ v(w)
parent = {e:e for e in disjoint}
def find(a):
    while parent[a]!=a:
        parent[a]=parent[parent[a]]
        a=parent[a]
    return a
def union(a,b):
    ra, rb = find(a), find(b)
    if ra!=rb: parent[rb]=ra

for w in W:
    union(("X", u[w]), ("Y", v[w]))

# Compute quotient classes = pushout
classes = {}
for e in disjoint:
    classes.setdefault(find(e), set()).add(e)

pushout = list(classes.values())
print("Pushout (X ⊔ Y)/~ has classes:")
for c in pushout:
    print(sorted(map(str,c)))


## Exercises

In [ ]:

# 1) Modify f and g and re-run to see how the pullback changes; verify projections commute.
# 2) Add another w2 in W that glues x0 with y1 and see new pushout quotient classes.
# 3) Prove (by code) the universal property: any cone to Z factors uniquely through the pullback.
